# Milestone Project 1: Food Vision Big

See the annoted version of this notebook on GitHub: https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/07_food_vision_milestone_project_1.ipynb

# Check GPU

Using mixed precision training we need a GPU with score 7.0+ (see here: https://developer.nvidia.com/cuda-gpus)

## Get helper functions 

In past modules, we've created a bunch of different helper functions to do small tasks required for our notebooks.

Rather than rewrite all of these, we can import a script and load them in from there.

The script we've got avialable can be found on GitHub: https://github.com/mrdbourke/tensorflow-deep-learning/blob/main/extras/helper_functions.py


In [1]:
# Download helper_function script
# !wget -P ./Extras/ https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/refs/heads/main/extras/helper_functions.py

In [2]:
# import series of helper functions for the notebook
from Extras.helper_functions import create_tensorboard_callback, plot_loss_curves, compare_historys

## Use Tensorflow Datasets to download data

If you want to get an overview of TensorFlow Datasets (tfds), read the guide: https://www.tensorflow.org/datasets

In [3]:
# get TensorFlow Datasets
import tensorflow as tf
import tensorflow_datasets as tfds

In [4]:
# List all available datasets
datasets_list = tfds.list_builders() # get all available datasets in TFDS
print("food101" in datasets_list) # is our target dataset available?

True


In [5]:
# Load in the data
(train_data, test_data), ds_info = tfds.load(name = "food101",
                                             split = ["train[:75%]", "train[25%:]"],
                                             shuffle_files = True,
                                             as_supervised = True, # Datasets gets returned in tuple format
                                             with_info = True,
                                             data_dir = "./Data")   

In [6]:
ds_info

tfds.core.DatasetInfo(
    name='food101',
    version=1.0.0,
    description='This dataset consists of 101 food categories, with 101'000 images. For each class, 250 manually reviewed test images are provided as well as 750 training images. On purpose, the training images were not cleaned, and thus still contain some amount of noise. This comes mostly in the form of intense colors and sometimes wrong labels. All images were rescaled to have a maximum side length of 512 pixels.',
    urls=['http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz'],
    features=FeaturesDict({
        'image': Image(shape=(None, None, 3), dtype=tf.uint8),
        'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
    }),
    total_num_examples=101000,
    splits={
        'train': 101000,
    },
    supervised_keys=('image', 'label'),
    citation="""@inproceedings{bossard14,
      title = {Food-101 -- Mining Discriminative Components with Random Forests},
      author = {Bossard, Lukas and Guilla

In [8]:
# features of Food101 from tfds
ds_info.features

FeaturesDict({
    'image': Image(shape=(None, None, 3), dtype=tf.uint8),
    'label': ClassLabel(shape=(), dtype=tf.int64, num_classes=101),
})

In [9]:
# get the class names 
class_names = ds_info.features["label"].names
class_names[:10]

['apple_pie',
 'baby_back_ribs',
 'baklava',
 'beef_carpaccio',
 'beef_tartare',
 'beet_salad',
 'beignets',
 'bibimbap',
 'bread_pudding',
 'breakfast_burrito']

## Explore the Food101 data from Tensorflow Dataset

To become one with data, we want to find:

* class names
* The shape of our input tensor (image tensor)
* The datatype of our input data
* What the labels look like (e.g are they one-hot-encoded or are they label encoded)
* Do the labels match up with the class names ?

In [11]:
# Take one sample of training data
train_one_sample = train_data.take(1) # samples are in the format (image_tensor, label)

In [13]:
# What does one sample of our data look like ?
train_one_sample

<TakeDataset element_spec=(TensorSpec(shape=(None, None, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>